## ASSUMPTION
This example is written assuming we have a service up and running for us to utilise

Copyright IBM Corp. 2022, 2023
@author James Strudwick IBM Research

## Dependencies

In [2]:
#For boc
from boc import BOC
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#For loading data
import pandas as pd
import numpy as np
# from io import StringIO

In [4]:
from time import sleep


## Utility function

In [5]:
def plot_progress(boc):
    exp_details = boc.get_experiment_details()['submittedVals']
    domain, targets = exp_details['X'], exp_details['y']

    plt.figure(figsize=(10,10))
    plt.plot(np.maximum.accumulate(targets), marker='o', linestyle='--', label='current best')
    plt.plot(targets, marker='*', linestyle='', color='r', label='epoch objective value')
#     plt.axhline(-1.0316, linestyle='--')
    plt.legend()
        
    targets = np.array(targets)
    X = np.array(domain)
    
    best_ind = np.argmax(targets)
    best_X = X[best_ind]
    best_y = targets[best_ind]
    
    plt.plot(np.maximum.accumulate(targets), marker='o', linestyle='--', color='b', label='current best')
    plt.plot(targets, marker='*', linestyle='', color='r', label='objective')
    plt.title(f'Best X found: {best_X}, best objectives found: {best_y}')
    plt.show()

## Data loading

In [6]:
exp_data=pd.read_csv("experiment_list_sterimol_final_iteration1_derivatives_no_const_and_energy.csv")

In [7]:
exp_data

,MCP dim1,MCP dim2,Catalyst dim1,Catalyst dim2,Solvent dim1,Solvent dim2,Cosolvent dim1,Cosolvent dim2,Additives dim1,Additives dim2,cosolvent vs solvent ratio,temp in degree celsius,cat. loading mol%,time in min,yield
0,-26.174568,1.631493,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,2.0,58
1,-26.174568,1.631493,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,4.0,61
2,-26.174568,1.631493,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,8.0,60
3,-26.174568,1.631493,1.216737,0.535949,-11.231260,-9.071500,0.000000,0.000000,0.0,0.0,0.00,21,10.0,2.0,0
4,-26.174568,1.631493,1.216737,0.535949,-11.231260,-9.071500,0.000000,0.000000,0.0,0.0,0.00,21,10.0,4.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,-26.174568,1.631493,2.618682,-13.172341,10.244377,7.708617,14.314211,-1.634429,0.0,0.0,3.33,21,10.0,2.0,0
172,-26.174568,1.631493,1.216737,0.535949,-6.429210,5.053837,14.314211,-1.634429,0.0,0.0,1.43,21,2.0,2.0,0
173,-17.221375,7.500541,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,2.0,43
174,-14.291092,-5.314128,1.216737,0.535949,-11.231260,-9.071500,14.314211,-1.634429,0.0,0.0,0.50,21,10.0,2.0,40


## Define the objective Function

In [8]:
# results=[]
def exp_1(X):
#     print(X)
    outputs=[]   
    dat=exp_data.to_numpy()
    
    for imp in X:
#         print(imp,type(imp))
        for line in dat:
#             print(line,type(line))
            if np.isclose(imp.astype(np.float64),line[:-1].astype(np.float64)).all():
                outputs.append([line[-1]])
#     results2.append(outputs)
    return np.array(outputs)

## Set up an experiment configuration

In [9]:
#surrogate
dim_types = ['continuous',#MCP dim1 
             'continuous',#MCP dim2
             'continuous',#Catalyst dim1 
             'continuous',#Catalyst dim2
             'continuous',#solvent dim1
             'continuous',#solvent dim2
             'continuous',#cosolvent dim1
             'continuous',#cosolvent dim2
             'continuous',#cosolvent vs solvent ratio
             'continuous',#additives dim1
             'continuous',#additives dim2
             'continuous',#temp in degree celsius
             'continuous',#cat. loading mol%
             'continuous',#time in hrs
            ]
domain=exp_data.drop('yield',axis=1).to_numpy().tolist()
acq_type = 'EI'
acq_kwargs = {'otypes': ['max'],  # list defining the objective types for each objective
              'normalise_x': True, # choice to normalise the domain values used by the model
              'normalise_y': True, # choice to normalise the target values used by the model
              'epsilon': 0.1, # explore-exploit tradeoff
              }

opt_types = {'continuous': 'Pool'} # set internal optimiser to  use initial 
                                                    # random sampling followed by cobyla

# parameters for the the Pool and cobyla optimiser
opt_kwargs = {'continuous':
                   {'domain':domain, 'dim_types': dim_types}
               }
# choice to use sequential single sampling 
sampler_type = 'Basic'
sampler_kwargs = {'nsample': 1}

# initial selection of 1 random sample and vary the random seed from 1 to 5 for repeatability
bo_kwargs = {'init_obs': 1,
             'log_level': 2,
             'seed': 2}

# model specification of two scikit-learn GPs for the two objectives
model_type = 'scikit-learn'
# selection of an ARD Matern kernel with nu 1.5 and initial lengthscale values of 0.1 for the
# models of each objective
model_kwargs = {'kernel': {'kernel_type' : 'Matern', 
                           'kernel_kwargs' : {}}}

# final config
config = {'model_type': model_type,
          'model_kwargs': model_kwargs,
          'acq_type': acq_type,
          'acq_kwargs': acq_kwargs,
          'dict_opt_types': opt_types,
          'dict_opt_kwargs': opt_kwargs,
          'sampler_type': sampler_type,
          'sampler_kwargs': sampler_kwargs,
          'bo_kwargs': bo_kwargs,
          }

## Instantiate the client

When setting `IP` you **do not** need to include `http://` or `https://` or the port number. The port number is set via `PORT`

In [10]:
# instantiate the BOC client
TOKEN = "00000000-0000-0000-0005-f987fc055425"
# TOKEN = "00000000-0000-0000-0005-f987ffa41104"
IP = "caplonsgprd-1.securegateway.appdomain.cloud"
PORT = "16551"
boc = BOC(user_token=TOKEN, ip=IP, port=PORT)
boc.validate_connection()

'Connection live.  boss version  -  2.0.2'

As the platform develops you might find that you get a warning similar to the one below if so we recomend that you up/down grade to a compatibile version of BOC

In [11]:
correct_ver = boc.__version__
boc.__version__='2.0.0'
boc.validate_connection()
correct_ver=boc.__version__

In [12]:
print(correct_ver,boc.__version__,boc.validate_connection())

2.0.0 2.0.0 Connection live.  boss version  -  2.0.2


## Submit an experiment

If you wish to wait until the experiement is ready for use, then set `wait` to `True`. If you wish to submit and not bother waiting, set it to `False`

In [15]:
# create the experiment
boc.generate_experiment(config, wait=True)

New experiment setup with id: 00000000-0000-0000-0006-030dc5af7f5d


KeyboardInterrupt: 

Store the experiment ID to be used later on

In [12]:
# boc.save_experiment_id('trial.yml')

To load previous experiment back in:

In [13]:
# boc.load_experiment_id('trial.yml')

## Experiment information

If you have multiple experiments or if you lose track of ids you can get a list of them by running:

In [14]:
boc.list_experiments(status=True)

No experiments registered for user.


[]

To restore experiment

If you want to know the status of your experiments as well then you can set `status` to `True`. If not you can set it to `False` or leave it out.

Then if you wish to switch to one of them you can set that by running:

In [20]:
boc.experiment_id = '00000000-0000-0000-0005-002217506420'

## Cumulative median test

In [17]:
list_backup=[]

In [18]:
def run(j):
    config['bo_kwargs']['seed']=j
    print('experiment ',j)
    boc.experiment_id=boc.generate_experiment(config, wait=True)
    sleep(2)
    boc.run_experiment(objective_f=exp_1, epochs=85)
    exp_details = boc.get_experiment_details()
    sleep(5)
    list1.append([i[0] for i in exp_details['submittedVals']['y']])
    list_backup.append([j,[i[0] for i in exp_details['submittedVals']['y']]])
    boc.archive_experiment()
    sleep(5) 
    print(boc.experiment_id,' is archived')

In [21]:
len(list1)

3

In [22]:
from time import sleep
# list1=[]
for j in range(4,41):
    if j==10 or j==20 or j==30 or j==40:
        print('Save data and clear experiments...')
        pd.DataFrame(list1).to_csv('raw%s-%s_with_derivatives.csv'%(1,j-1))
        l = [x[0] for x in boc.list_experiments(status=True)]
        for x in l:
            boc.experiment_id = x
            boc.kill_experiment()
            sleep(5)
            print(x,' is deleted')
        run(j)
    else:
        run(j)


experiment  4
New experiment setup with id: 00000000-0000-0000-0006-031c0725cb11
Waiting for service to be created (check 7)
Experiment ready. bowi version 1.0.6
epoch: 0
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  -6.42920977
    5.05383681  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[0.]]

epoch: 1
 X: [[-26.17456797   1.63149257   0.           0.         -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           4.        ]]
 y: [[0.]]

epoch: 2
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  17.10765351
    0.25635035  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           8.        ]]
 y: [[24.]]

epoch: 3
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  10.2443772
    7.70861653  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           8.        ]]
 y: [[26.]]

epoch: 4
 

epoch: 38
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895  35.81939383   5.36416106
    0.5         21.          10.           2.        ]]
 y: [[30.]]

epoch: 39
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976 -16.12391177 -14.68563103   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[0.]]

epoch: 40
 X: [[-26.17456797   1.63149257  -0.40451351   0.70107669 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           8.        ]]
 y: [[37.]]

epoch: 41
 X: [[-26.17456797   1.63149257 -10.58822511  -2.0722261  -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           2.        ]]
 y: [[1.]]

epoch: 42
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -26.10750752
  -16.58978902  14.31421122  -1.63442895   0.           0.
    0.

epoch: 76
 X: [[-26.17456797   1.63149257  10.03831124   2.72251139 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           8.        ]]
 y: [[16.]]

epoch: 77
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976 -21.79041995  -7.46693486   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[0.]]

epoch: 78
 X: [[-26.17456797   1.63149257 -15.62634673   9.61611704 -11.2312595
   -9.07149976  14.31421122  -1.63442895  -5.02749884  -9.87145008
    0.5         21.          10.           2.        ]]
 y: [[3.]]

epoch: 79
 X: [[-26.17456797   1.63149257  -0.40451351   0.70107669 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           4.        ]]
 y: [[38.]]

epoch: 80
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  -6.42920977
    5.05383681  14.31421122  -1.63442895   0.           0.
    0.5       

epoch: 29
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.         110.          24.        ]]
 y: [[0.]]

epoch: 30
 X: [[-26.17456797   1.63149257 -15.45709194  18.60670191 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           4.        ]]
 y: [[3.]]

epoch: 31
 X: [[-26.17456797   1.63149257 -10.58822511  -2.0722261  -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           2.        ]]
 y: [[1.]]

epoch: 32
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.         110.           4.        ]]
 y: [[0.]]

epoch: 33
 X: [[-2.61745680e+01  1.63149257e+00  1.21673670e+00  5.35948542e-01
  -1.12312595e+01 -9.07149976e+00  1.43142112e+01 -1.63442895e+00
   0.00000

epoch: 68
 X: [[-26.17456797   1.63149257  10.03831124   2.72251139 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           4.        ]]
 y: [[20.]]

epoch: 69
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         10.          10.           2.        ]]
 y: [[32.]]

epoch: 70
 X: [[-26.17456797   1.63149257 -15.62634673   9.61611704 -11.2312595
   -9.07149976  14.31421122  -1.63442895  -5.02749884  -9.87145008
    0.5         21.          10.           2.        ]]
 y: [[3.]]

epoch: 71
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.           1.           4.        ]]
 y: [[10.]]

epoch: 72
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  10.2443772
    7.70861653  14.31421122  -1.63442895   0.           0.
    0.5         21.   

...

epoch: 15
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  10.2443772
    7.70861653  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           4.        ]]
 y: [[25.]]

epoch: 16
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  17.10765351
    0.25635035  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           4.        ]]
 y: [[23.]]

epoch: 17
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  17.10765351
    0.25635035  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[23.]]

epoch: 18
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854  51.05607948
  -17.74595545  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[13.]]

epoch: 19
 X: [[-26.17456797   1.63149257   2.61868239 -13.17234088 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.       

epoch: 54
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895  15.89802239  -7.73504093
    0.5         21.          10.           8.        ]]
 y: [[23.]]

epoch: 55
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  -8.36486734  28.17894833   0.           0.
    0.5         21.          10.           8.        ]]
 y: [[24.]]

epoch: 56
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -12.15940052
   -2.93111641  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           4.        ]]
 y: [[20.]]

epoch: 57
 X: [[-26.17456797   1.63149257 -10.41032386 -11.93316611 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           8.        ]]
 y: [[2.]]

epoch: 58
 X: [[-26.17456797   1.63149257  -0.40451351   0.70107669 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0

epoch: 5
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.           0.5          8.        ]]
 y: [[3.]]

epoch: 6
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.         110.           8.        ]]
 y: [[3.]]

epoch: 7
 X: [[-26.17456797   1.63149257  10.03831124   2.72251139 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[19.]]

epoch: 8
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          50.          24.        ]]
 y: [[6.]]

epoch: 9
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         10.          10.     

epoch: 44
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -11.03618398  15.59494344
    0.5         21.          10.           4.        ]]
 y: [[54.]]

epoch: 45
 X: [[-26.17456797   1.63149257   1.2167367    0.53594854 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          50.           2.        ]]
 y: [[5.]]

epoch: 46
 X: [[-26.17456797   1.63149257  -0.40451351   0.70107669 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           2.        ]]
 y: [[38.]]

epoch: 47
 X: [[-26.17456797   1.63149257  -0.40451351   0.70107669 -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.91313079
    0.5         21.          10.           8.        ]]
 y: [[37.]]

epoch: 48
 X: [[-26.17456797   1.63149257 -10.58822511  -2.0722261  -11.2312595
   -9.07149976  14.31421122  -1.63442895 -14.04107488  -7.9131307

epoch: 82
 X: [[-2.61745680e+01  1.63149257e+00 -8.45321623e+00 -3.86909291e+00
  -1.12312595e+01 -9.07149976e+00  1.43142112e+01 -1.63442895e+00
   2.22044605e-16  0.00000000e+00  5.00000000e-01  2.10000000e+01
   1.00000000e+01  2.00000000e+00]]
 y: [[12.]]

epoch: 83
 X: [[-2.61745680e+01  1.63149257e+00  1.21673670e+00  5.35948542e-01
  -1.12312595e+01 -9.07149976e+00  1.43142112e+01 -1.63442895e+00
   0.00000000e+00  1.11022302e-16  5.00000000e-01  2.10000000e+01
   2.50000000e+01  2.40000000e+01]]
 y: [[24.]]

epoch: 84
 X: [[-26.17456797   1.63149257  13.17525075  -5.84847326 -11.2312595
   -9.07149976  14.31421122  -1.63442895   0.           0.
    0.5         21.          10.           2.        ]]
 y: [[27.]]

00000000-0000-0000-0006-0360c473728e  is archived


In [23]:
pd.DataFrame(list1).to_csv('raw%s-%s_with_derivatives.csv'%(0,39))

In [54]:
list_backup

[[35,
  [12.0,
   23.0,
   25.0,
   19.0,
   0.0,
   24.0,
   40.0,
   0.0,
   25.0,
   0.0,
   32.0,
   38.0,
   23.0,
   22.0,
   27.0,
   0.0,
   5.0,
   0.0,
   3.0,
   25.0,
   25.0,
   37.0,
   0.0,
   5.0,
   21.0,
   1.0,
   31.0,
   29.0,
   24.0,
   2.0,
   3.0,
   0.0,
   0.0,
   17.0,
   24.0,
   23.0,
   1.0,
   37.0,
   5.0,
   0.0,
   0.0,
   16.0,
   1.0,
   2.0,
   0.0,
   0.0,
   0.0,
   0.0,
   25.0,
   0.0,
   0.0,
   3.0,
   0.0,
   22.0,
   5.0,
   3.0,
   0.0,
   4.0,
   41.0,
   2.0,
   53.0,
   1.0,
   3.0,
   16.0,
   30.0,
   22.0,
   1.0,
   3.0,
   23.0,
   0.0,
   16.0,
   0.0,
   6.0,
   2.0,
   23.0,
   23.0,
   26.0,
   1.0,
   35.0,
   0.0,
   20.0,
   7.0,
   25.0,
   28.0,
   0.0,
   37.0,
   0.0,
   22.0,
   5.0,
   3.0,
   0.0,
   24.0,
   61.0,
   26.0,
   28.0,
   32.0,
   54.0,
   27.0,
   29.0,
   27.0]],
 [36,
  [0.0,
   61.0,
   0.0,
   16.0,
   38.0,
   19.0,
   5.0,
   0.0,
   12.0,
   23.0,
   23.0,
   32.0,
   26.0,
   2.0,
   23.0,
   30

In [ ]:
config['bo_kwargs']['seed']=j
print('experiment ',j)
boc.experiment_id=boc.generate_experiment(config, wait=True)
boc.run_experiment(objective_f=exp_1, epochs=100)
exp_details = boc.get_experiment_details()
list1.append([i[0] for i in exp_details['submittedVals']['y']])
boc.archive_experiment()
sleep(5) 
print(boc.experiment_id,' is archived')

In [46]:
boc.experiment_id = '00000000-0000-0000-0006-002217506420'
exp_details = boc.get_experiment_details()
exp_details

C:\Users\cc2819\anaconda3\envs\boc\lib\site-packages\boc\boc.py:318: UserWarning: Memory at 48690.78%, please consider contacting your admin to upgrade the resources
  warnings.warn(resource_warning)


{'cachedValuesToReturn': [[1.186137917,
   0.634095202,
   -11.21819749,
   -9.225527544,
   14.36486742,
   -1.75190403,
   0.5,
   0.0,
   0.0,
   21.0,
   0.5,
   24.0]],
 'config': {'acq_kwargs': {'epsilon': 0.1,
   'normalise_x': True,
   'normalise_y': True,
   'otypes': ['max']},
  'acq_type': 'EI',
  'bo_kwargs': {'init_obs': 1, 'log_level': 2, 'seed': 32},
  'dict_opt_kwargs': {'continuous': {'dim_types': ['continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous'],
    'domain': [[1.186137917,
      0.634095202,
      -11.21819749,
      -9.225527544,
      14.36486742,
      -1.75190403,
      0.5,
      0.0,
      0.0,
      21.0,
      10.0,
      2.0],
     [1.186137917,
      0.634095202,
      -11.21819749,
      -9.225527544,
      14.36486742,
      -1.75190403,
      0.5,
      0.0,
      0.0,
      21.0,
 

In [47]:
boc.experiment_id = '00000000-0000-0000-0006-002217506420'
exp_details = boc.get_experiment_details()
list32=[]
list32.append([i[0] for i in exp_details['submittedVals']['y']])

C:\Users\cc2819\anaconda3\envs\boc\lib\site-packages\boc\boc.py:318: UserWarning: Memory at 48690.78%, please consider contacting your admin to upgrade the resources
  warnings.warn(resource_warning)


In [48]:
pd.set_option('display.max_columns', None)
pd.DataFrame(list32).to_csv('raw32.csv')

In [27]:
exp_details = boc.get_experiment_details()


In [28]:
exp_details

{'message': 'Either token/experiment does not exist or the token does not have access to the given token'}

In [19]:
list6_to_13=[]
l = [x[0] for x in boc.list_experiments(status=True)]
for x in l:
    boc.experiment_id = x
    exp_details = boc.get_experiment_details()
    list6_to_13.append([i[0] for i in exp_details['submittedVals']['y']])

Multiple Ids retrieved.


C:\Users\cc2819\anaconda3\envs\boc\lib\site-packages\boc\boc.py:318: UserWarning: Memory at 38446.02%, please consider contacting your admin to upgrade the resources
  warnings.warn(resource_warning)
C:\Users\cc2819\anaconda3\envs\boc\lib\site-packages\boc\boc.py:318: UserWarning: Memory at 2956.56%, please consider contacting your admin to upgrade the resources
  warnings.warn(resource_warning)


To See the experiment records and actual errors:


In [22]:
print(boc.get_experiment_logs())

experiment is not yet ready, current status: Archived. Please check back later
None


### Check the state of the experiment

In [18]:
exp_details = boc.get_experiment_details()


In [19]:
exp_details

{'cachedValuesToReturn': [],
 'config': {'acq_kwargs': {'epsilon': 0.1,
   'normalise_x': True,
   'normalise_y': True,
   'otypes': ['max']},
  'acq_type': 'EI',
  'bo_kwargs': {'init_obs': 1, 'log_level': 2, 'seed': 0},
  'dict_opt_kwargs': {'continuous': {'dim_types': ['continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous',
     'continuous'],
    'domain': [[0.33361911,
      0.595589152,
      -3.103239204,
      -2.07761934,
      4.727547618,
      -0.758874265,
      0.5,
      0.0,
      0.0,
      21.0,
      10.0,
      2.0],
     [0.33361911,
      0.595589152,
      -3.103239204,
      -2.07761934,
      4.727547618,
      -0.758874265,
      0.5,
      0.0,
      0.0,
      21.0,
      10.0,
      4.0],
     [0.33361911,
      0.595589152,
      -3.103239204,
      -2.07761934,
      4.727547618,
      -0.758874265,
      0.5,

Now archive the experiment and wait for the archiving process to complete

In [65]:
from time import sleep
l = [x[1] for x in boc.list_experiments(status=True)]
stat = (l[0]=='Archived')
#If only 1 experiment, reference 5-8 lines
while not stat:
    sleep(5)
    l = [x[1]=='Ready' for x in boc.list_experiments(status=True)]
    stat = (l[0]=='Archived')
boc.list_experiments(status=True)

Multiple Ids retrieved.
Multiple Ids retrieved.


[['00000000-0000-0000-0005-fc63ddf7ebbf', 'Archived'],
 ['00000000-0000-0000-0005-fc640cdec244', 'Ready'],
 ['00000000-0000-0000-0005-fc641fb59a68', 'ArchivingInProgress']]

With it archived we can still check the details of the experiment

At a later date we can then restore it

In [71]:
from time import sleep
stat = all([x[1]=='Ready' for x in boc.list_experiments(status=True)])
# while not stat:
#     sleep(5)
#     stat = all([x[1]=='Ready' for x in boc.list_experiments(status=True)])
boc.list_experiments(status=True)

Multiple Ids retrieved.
Multiple Ids retrieved.


[['00000000-0000-0000-0005-fc63ddf7ebbf', 'Archived'],
 ['00000000-0000-0000-0005-fc640cdec244', 'Ready'],
 ['00000000-0000-0000-0005-fc641fb59a68', 'Restoring']]

Now that the experiment is restored the `get_next_X` values should be the same/similar to those that we given just before we archived the experiment

In [42]:
boc.get_next_X()

array([[ 0.235925  , -0.69554882]])

### To RESTORE experiment

In [19]:
boc.experiment_id = '00000000-0000-0000-0005-0026067429ba'
boc.restore_experiment()

Begginning Restoration process


### To KILL experiment

In [17]:
boc.list_experiments(status=True)

Multiple Ids retrieved.


[['00000000-0000-0000-0006-002217506420', 'Ready'],
 ['00000000-0000-0000-0006-00236b0df1bc', 'Ready'],
 ['00000000-0000-0000-0006-0023c4bd979a', 'Ready'],
 ['00000000-0000-0000-0006-00258296f6e9', 'Ready'],
 ['00000000-0000-0000-0006-0026067429ba', 'Ready']]

In [16]:
#to kill all experiments
from time import sleep
l = [x[0] for x in boc.list_experiments(status=True)]
for x in l:
    boc.experiment_id = x
    boc.kill_experiment()
    sleep(5)
    print(x,' is deleted')

Multiple Ids retrieved.
00000000-0000-0000-0006-030dc5af7f5d  is deleted


In [19]:
boc.experiment_id = '00000000-0000-0000-0006-0026067429ba'
boc.kill_experiment()

'Experiment is being deleted'

### To ARCHIVE experiment

In [28]:
boc.experiment_id = '00000000-0000-0000-0005-ffe0fb19f55d'
# exp_details = boc.get_experiment_details()
# len(exp_details['submittedVals']['y'])
boc.archive_experiment()

'Experiment is being archived'

In [ ]:
from time import sleep
l = [x[1] for x in boc.list_experiments(status=True)]
stat = (l[0]=='Archived')
while not stat:
    sleep(5)
    l = [x[1]=='Archived' for x in boc.list_experiments(status=True)]
    stat = (l[0]=='Archived')
boc.list_experiments(status=True)

In [20]:
boc.experiment_id = '00000000-0000-0000-0005-0023c4bd979a'
boc.get_experiment_details()

{'message': 'Either token/experiment does not exist or the token does not have access to the given token'}

In [13]:
boc.list_experiments(status=True)

Multiple Ids retrieved.


[['00000000-0000-0000-0006-030a82dc9ce8', 'Archived'],
 ['00000000-0000-0000-0006-030b2e12d212', 'Archived']]

In [118]:
boc.archive_experiment()

'Either token/experiment does not exist or the token does not have access to the given token'

If archived, unable to be deleted; 